# Similarity Models — Architecture, Time Complexity & Optimizations

This notebook documents the Tier 1 similarity engine for the Medical Safety Assistant project.
It covers **what** the models are, **why** they are implemented the way they are, their **time complexity**, and every **optimization** applied.

---

## 1. The Three-File Architecture

The similarity system is split across three files. Each has a single, clearly defined responsibility:

```
src/models/tier_1_similarity/
 ├── binary_similarity.py   ← The math engine (one formula, two named consumers)
 ├── chemical_sim.py        ← Backwards-compatible chemical API wrapper
 └── advanced_fusion.py     ← Data orchestrator (loading, caching, fusion)
```

**Why split it this way?**  
This follows the **Single Responsibility Principle**: each file has exactly one reason to change.  
- If the math formula changes → edit `binary_similarity.py` only.  
- If the fingerprint generation changes → edit `advanced_fusion.py` only.  
- If you rename the public API → edit `chemical_sim.py` only.  

No change to one component should require touching the others.

## 2. The Models

### 2a. `BinarySimilarityEngine` (base class)

Both chemical **Tanimoto** and phenotypic **Jaccard** are mathematically identical:

$$T = J = \frac{c}{a + b + c} = \frac{|A \cap B|}{|A \cup B|}$$

Where:
- **c** = bits set to `1` in **both** vectors (intersection)  
- **a** = bits set to `1` in A but not B  
- **b** = bits set to `1` in B but not A  

Rather than duplicating this formula, `BinarySimilarityEngine._calculate_binary_similarity()` implements it once using **Sparse CSR matrix algebra**.

### 2b. `TanimotoEngine` (chemical fingerprints)

A named subclass of `BinarySimilarityEngine`. Applied to **2048-bit Morgan fingerprints** that encode a molecule's chemical structure.

### 2c. `JaccardEngine` (phenotypic side-effects)

A named subclass of `BinarySimilarityEngine`. Applied to **SIDER side-effect profiles** — binary rows indicating which side effects each drug causes.

### 2d. `SimilarityEngine` (backwards-compatible wrapper)

A thin wrapper in `chemical_sim.py` that exposes `SimilarityEngine.calculate_tanimoto()` for all existing callers without requiring them to import from the new `binary_similarity` module.

### 2e. `AdvancedFusionModel` (the orchestrator)

Combines the two signals into a weighted fusion score:

$$\text{Fusion Score} = \begin{cases} s_{\text{chem}} & \text{if } s_{\text{pheno}} = 0 \\ w \cdot s_{\text{pheno}} + (1-w) \cdot s_{\text{chem}} & \text{otherwise} \end{cases}$$

The optimal weight `w = 1.0` was determined empirically by grid search over 500 real drug interactions, maximising **AUPR**.

## 3. Time Complexity

### Dense Matrix Baseline (original approach)

Computing Tanimoto with standard `numpy.dot`:

| Operation | Complexity |
|-----------|------------|
| Dot product (N drugs × M drugs, F features) | **O(N × M × F)** |
| For 1 drug vs 1000 drugs (F=2048) | 2,048,000 multiplications |

Problem: Morgan fingerprints are ~97.5% zeroes. The dense multiplication evaluates 2,048,000 equations even though 97.5% of them are `0 × 0 = 0`.

### Sparse CSR Matrix (current approach)

| Operation | Complexity |
|-----------|------------|
| Sparse dot product (nnz = ~50 set bits per drug) | **O(N × M × nnz²)** |
| For 1 drug vs 1000 drugs (nnz≈50) | ~50,000 operations — **~40× less** |

**Space complexity:**

| Structure | Memory for 10,000 drugs |
|-----------|-------------------------|
| Dense float matrix (2048 features) | `10,000 × 2048 × 8 bytes ≈ 163 MB` |
| CSR sparse matrix (50 non-zeros) | `10,000 × 50 × 8 bytes ≈ 4 MB` — **~97% reduction** |

## 4. Architecture Decisions & Rationale

### Decision 1: Base Class Pattern (DRY)

**Problem:** Tanimoto and Jaccard are the same formula. Writing it twice violates DRY (Don't Repeat Yourself).

**Solution:** `BinarySimilarityEngine` owns one implementation. `TanimotoEngine` and `JaccardEngine` are named subclasses — they add semantic clarity without duplicating logic.

---

### Decision 2: Data Extraction stays in the Orchestrator

**Problem:** Should `binary_similarity.py` parse SMILES strings and generate fingerprints itself?

**Solution:** No. `binary_similarity.py` only accepts plain numpy arrays. RDKit parsing, fingerprint generation, and caching all live in `AdvancedFusionModel`. This keeps the math engine pure and reusable for non-chemistry data.

**Why it matters:** If you use `TanimotoEngine` for a different domain (genetic sequences, user preference vectors), it works out of the box — no chemistry library dependency.

---

### Decision 3: O(1) Phenotypic Lookup Map

**Problem:** Accessing rows from a Pandas DataFrame by drug ID (`df.loc[id]`) is `O(log n)` per lookup — acceptable for a few drugs, slow for thousands.

**Solution:** At `__init__` time, extract the DataFrame index into a Python dict:
```python
self.pheno_idx_map = {drug_id: idx for idx, drug_id in enumerate(df_pheno.index)}
```
Dict lookup is `O(1)`. The matrix becomes a CSR sparse structure indexed by integer position.

---

### Decision 4: Vectorized API

**Problem:** Calling `predict_fusion_score()` for each drug pair one at a time requires a Python `for` loop over 500 pairs — bypassing all the numpy/CSR speed benefits.

**Solution:** All methods accept **lists of drugs** and process them in one batched matrix call. The grid search runs 500 pairs in a single vectorized operation.

## 5. Optimizations Applied

### Optimization 1: Sparse CSR Matrix Multiplication

**Before:** `numpy.dot(A, B.T)` — dense multiplication, evaluates all zeros.

**After:** `csr_matrix(A).dot(csr_matrix(B).T)` — skips all zero entries.

| Metric | Dense | Sparse CSR |
|--------|-------|------------|
| Operations per pair | 2,048,000 | ~50,000 |
| Theoretical speedup | — | ~40× |
| Grid search time (500 pairs) | baseline | **0.34s** |

---

### Optimization 2: Fingerprint Cache (compute once, reuse forever)

**Before:** Each call to `get_chemical_similarity(aspirin, drug_X)` re-parsed the Aspirin SMILES and re-generated its Morgan fingerprint.

**After:** `_fp_cache` stores the result after the first parse. Repeated comparisons involving the same drug do zero RDKit work.

---

### Optimization 3: MorganGenerator API (RDKit new API)

**Before (deprecated):**
```python
fp_obj = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)  # RDKit BitVect object
fp_arr = np.zeros((1,))
DataStructs.ConvertToNumpyArray(fp_obj, fp_arr)                      # explicit conversion step
```

**After (current):**
```python
generator = AllChem.GetMorganGenerator(radius=2, fpSize=2048)
fp_arr = generator.GetFingerprintAsNumPy(mol)                        # numpy array directly
```

**Results:**
- ✅ Deprecation warnings eliminated  
- ✅ `DataStructs` import removed  
- ✅ Grid search time: `0.68s → 0.34s` (2× faster fingerprint generation)

---

### Optimization 4: Vectorized Evaluation Script

**Before (`tune_fusion_weights.py`):**
```python
for _, row in full_data.iterrows():          # Python for-loop
    c_score = model.get_chemical_similarity(row['smiles1'], row['smiles2'])
    p_score = model.get_phenotypic_similarity(row['id1'], row['id2'])
```

**After:**
```python
s_chem_arr  = model.get_chemical_similarity(smiles_A, smiles_B)   # one call for all 500 pairs
s_pheno_arr = model.get_phenotypic_similarity(ids_A, ids_B)       # one call for all 500 pairs
```

Eliminates 500 Python function call overheads and feeds one large matrix to C-level numpy code.

## 6. Live Demo — Running the Models

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
from src.models.tier_1_similarity.binary_similarity import TanimotoEngine, JaccardEngine

In [ ]:
# --- Tanimoto Demo ---
print("=== TanimotoEngine — Chemical Fingerprint Similarity ===")

identical  = np.array([[1, 1, 0, 0, 1]])
disjoint   = np.array([[0, 0, 1, 1, 0]])
half_overlap = np.array([[1, 0, 0, 0, 1]])

query = np.array([[1, 1, 0, 0, 1]])

scores = TanimotoEngine.calculate(
    query_fingerprints=query,
    database_fingerprints=np.vstack([identical, disjoint, half_overlap])
)

print(f"vs Identical:     {scores[0][0]:.3f}  (expected: 1.000)")
print(f"vs Disjoint:      {scores[0][1]:.3f}  (expected: 0.000)")
print(f"vs Half Overlap:  {scores[0][2]:.3f}  (expected: 0.667)")

In [ ]:
# --- Jaccard Demo ---
print("=== JaccardEngine — Phenotypic Side-Effect Similarity ===")

# Simulated side-effect profiles (binary: 1 = causes this side effect)
drug_A = np.array([[1, 1, 0, 1, 0, 1, 0, 0]])
drug_B = np.array([[1, 1, 0, 1, 0, 0, 0, 0]])  # shares 3 of 4 side effects
drug_C = np.array([[0, 0, 1, 0, 1, 0, 1, 1]])  # shares no side effects

scores = JaccardEngine.calculate(
    query_phenotype_vectors=drug_A,
    database_phenotype_vectors=np.vstack([drug_B, drug_C])
)

print(f"Drug A vs Drug B (high overlap): {scores[0][0]:.3f}")
print(f"Drug A vs Drug C (no overlap):   {scores[0][1]:.3f}")

In [ ]:
# --- Fusion Demo ---
print("=== AdvancedFusionModel — Full Pipeline ===")

from src.models.tier_1_similarity.advanced_fusion import AdvancedFusionModel

model = AdvancedFusionModel()

aspirin  = {'id': 'DB00945', 'smiles': 'O=C(C)Oc1ccccc1C(=O)O'}
warfarin = {'id': 'DB00682', 'smiles': 'CC(=O)CC(c1ccccc1)c1c(O)c2ccccc2oc1=O'}

c_score = model.get_chemical_similarity(aspirin['smiles'], warfarin['smiles'])
p_score = model.get_phenotypic_similarity(aspirin['id'], warfarin['id'])
f_score = model.predict_fusion_score(aspirin, warfarin)

print(f"Chemical Similarity  (Tanimoto): {float(c_score):.4f}")
print(f"Phenotypic Similarity (Jaccard): {float(p_score):.4f}")
print(f"Fusion Score:                    {float(f_score):.4f}")

## 7. Summary

| Component | Role | Key Design Choice |
|-----------|------|-------------------|
| `BinarySimilarityEngine` | Sparse CSR intersection-over-union math | One formula, two named consumers |
| `TanimotoEngine` | Chemical similarity | Named subclass — no code duplication |
| `JaccardEngine` | Phenotypic similarity | Named subclass — no code duplication |
| `SimilarityEngine` | Backwards-compatible wrapper | Legacy API preserved |
| `AdvancedFusionModel` | Data loading, caching, orchestration | Strict separation from math engine |

| Optimization | Benefit |
|--------------|--------|
| Sparse CSR matrices | ~40× fewer operations per pairwise comparison |
| Fingerprint caching | Each drug SMILES parsed exactly once per session |
| MorganGenerator API | 2× faster fingerprint generation, zero deprecation warnings |
| Vectorized batch API | 500 drug pair scores in a single C-level matrix call |

## 8. Scientific Evaluation — Ablation Study

To rigorously prove the value of the Tier 1 models, we evaluate them against a ground truth dataset of known Drug-Drug Interactions (DDIs) using a **5-seed Ablation Study**.

### Ground Truth
- **Positive Label ($y=1$)**: Two drugs have a documented interaction in the DrugBank database.
- **Negative Label ($y=0$)**: A randomly sampled pair of drugs with no documented interaction. We use a **1:10 positive-to-negative ratio**, which mimics the extreme class imbalance of real-world drug discovery where most compounds do not interact.

---

### Evaluation Metrics

Because DDI prediction is highly imbalanced, standard accuracy is misleading. We rely on three industry-standard metrics:

1. **AUPR (Area Under Precision-Recall Curve) — Primary Metric**
   Evaluates the trade-off between Precision (confidence in positive predictions) and Recall (ability to find all true interactions). It is the gold standard for imbalanced datasets.

2. **Enrichment Factor at 1% (EF@1%)**
   A pharmaceutical screening metric. It asks: *"If a scientist only investigates the top 1% of the model's highest-scored predictions, how many more true DDIs will they find compared to picking pairs at random?"*
   - EF = 1.0 means the model is guessing randomly.
   - EF = 7.5 means the top 1% is 7.5× richer in real interactions than random chance.

3. **AUROC (Area Under Receiver Operating Characteristic)**
   Measures the model's ability to rank a random positive pair higher than a random negative pair across all possible thresholds.

### The Ablation Strategy

We isolate each predictive modality and compare it against the fused model using the exact same evaluation pairs. To ensure statistical significance, this is repeated across **5 random seeds** (drawing different negative samples each time) to produce a `mean ± std`.

| Condition | Model | Source Data |
|-----------|-------|-------------|
| **A** | `TanimotoEngine` | Chemical Structure (Morgan Fingerprints) |
| **B** | `JaccardEngine` | Phenotypic Profile (SIDER Side-Effects) |
| **C** | `AdvancedFusionModel` | Multi-modal Fusion ($w=1.0$) |

### Results & Interpretation

Based on the evaluation run over 380,000 pairs (6,981 positive, 69,810 negative per seed):

```text
Condition                 AUPR                 AUROC         Avg_Precision                 EF@1%
------------------------------------------------------------------------------------------------
A: Chemical Only          0.1142 ± 0.0004      0.5848 ± 0.0011     0.1142 ± 0.0004       1.3637 ± 0.0141
B: Phenotypic Only        0.3691 ± 0.0045      0.5917 ± 0.0003     0.2069 ± 0.0021       7.5376 ± 0.0797
C: Fused                  0.1445 ± 0.0004      0.6194 ± 0.0010     0.1444 ± 0.0004       3.0024 ± 0.0431
```

**Scientific Takeaways:**

1. **Phenotypic Dominance in Top-K (Precision)**: Condition B heavily outperforms in AUPR and EF@1%. In the top 1% of predictions, side-effect overlap yields **7.5× more real interactions** than random chance (compared to 1.3× for chemical structure). This suggests that drugs exhibiting similar side effects in humans are overwhelmingly likely to share unmapped pharmacological pathways.
2. **Chemical Broad-Scale Signal (Recall)**: While poor at top-1% precision, chemical structural overlap contributes to a broader baseline awareness, helping the **Fused Model (Condition C)** win the overall AUROC metric (0.6194).
3. **Conclusion**: Relying purely on 2D chemical structure is insufficient for modern DDI detection. The phenotypic layer acts as an essential proxy for in-vivo bioactivity, capturing drug relationships that molecular graphs alone miss.

## 9. Baseline Evaluation Conclusion: The Limits of Linear Fusion

This 5-seed ablation study establishes the baseline performance for the Tier 1 similarity models. The results highlight a fundamental limitation in our current architecture:

1. **Modal Dominance**: The phenotypic modality (`JaccardEngine`) heavily dominates precision (AUPR $0.3691$, EF@1% $7.54$), while the chemical modality (`TanimotoEngine`) struggles to rank true interactions highly on its own (AUPR $0.1142$).
2. **The Fusion Ceiling**: The `AdvancedFusionModel` attempts to bridge these domains using a simple linear weighted sum. However, this naive approach fails to create synergy. The fused model achieves an AUROC of $0.6194$ (a slight improvement in global ranking), but its AUPR ($0.1445$) collapses compared to using the phenotypic data alone.

### Why does Linear Fusion fail here?
A linear sum assumes that a unit of "chemical similarity" is directly interchangeable with a unit of "side-effect similarity". Biologically, this is false. A drug's 2D chemical structure and its systemic human side-effects exist across vastly different dimensional scales. 

When the model simply adds these scores together, the noisy, low-precision chemical signal dilutes the highly predictive phenotypic signal, dragging down the top-K enrichment (EF@1% drops from $7.54$ to $3.00$).

---

### Progression to Tier 2: Heterogeneous Information Networks
These baseline metrics prove that **linear weighted fusion is insufficient for bridging distinct biological domains**. 

To accurately predict Drug-Drug Interactions, we cannot simply add disjoint similarity scores together. We must model *how* these domains interact. This necessitates the development of the **Tier 2 Architecture**: mapping drugs, protein targets, and biological pathways into a single Heterogeneous Information Network (HIN) and using advanced graph-walking algorithms (Random Walk with Restart) to naturally uncover the non-linear topological relationships between them.

## 10. Tier 2 Breakthrough: The Protein-Protein Interaction (PPI) Network

Following the failure of linear fusion to bridge disjoint domains, we transitioned to a graph-based approach. 

In **Phase 1 of Tier 2**, we implemented a Random Walk with Restart (RWR) algorithm over the Homogeneous PPI graph. Instead of looking at side-effects or chemical vectors, we mapped each drug to its known protein targets, and simulated how a signal diffuses through the interactome (the known network of how human proteins interact with each other).

To solve the critical computational bottleneck of graph walking, we **precomputed** the steady-state biological footprints for all 1,500 drugs into a continuous sparse matrix, reducing real-time similarity to a lightning-fast $O(1)$ Cosine Similarity lookup.

### Results: Tier 2 vs Tier 1

When we drop the vectorized Tier 2 model into the exact same 380,000-pair 5-seed ablation study, the results are scientifically profound:

```text
Condition                 AUPR                 AUROC         Avg_Precision                 EF@1%
------------------------------------------------------------------------------------------------
A: Chemical Only          0.1142 ± 0.0004      0.5848 ± 0.0011     0.1142 ± 0.0004       1.3637 ± 0.0141
B: Phenotypic Only        0.3691 ± 0.0045      0.5917 ± 0.0003     0.2069 ± 0.0021       7.5376 ± 0.0797
C: Tier 1 Fused           0.1445 ± 0.0004      0.6194 ± 0.0010     0.1444 ± 0.0004       3.0024 ± 0.0431
D: Tier 2 PPI RWR         0.3921 ± 0.0017      0.7599 ± 0.0002     0.3447 ± 0.0017       7.5548 ± 0.1599
```

### Scientific Takeaway

The Tier 2 PPI network absolutely shatters the Tier 1 baseline.

- **AUROC Leap**: It jumps from $0.6194$ (Tier 1 Fused) up to **$0.7599$**. This proves that tracking the structural paths between protein targets in the interactome handles global drug ranking vastly better than chemical or phenotypic overlap.
- **Precision Recovery**: It achieves an AUPR of $0.3921$, reclaiming and ultimately exceeding the ultra-high precision that the Tier 1 Phenotypic model exhibited ($0.3691$), without suffering from the "Fusion Ceiling" dilution.

The graph topology naturally bridges the biological domain. However, we are currently only using Protein-to-Protein edges. The final evolutionary step is introducing the Heterogeneous Information Network (HIN).